In [ ]:
import os
import sys

# для импорта модулей из корня проекта
sys.path.append('../')

import pandas as pd
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from bert_score import score
from langchain_community.vectorstores import FAISS


from retrieval.doc_loader import parse_html_with_langchain
from retrieval.vectorstore import create_vectorstore
from embeddings.bge_embeddings import BGEEmbeddings
from model.llm_setup import get_llm
from model.rag_chain import create_rag_chain


/home/user12/miniconda3/envs/gigachat/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
pd.set_option('display.max_colwidth', 20)

In [ ]:
file_path = "../data/TestDataset.csv"
data = pd.read_csv(file_path, sep=";")
data_dir = "../data"
index_path = "../faiss"

In [11]:
print("Пример данных:")
data.head()

Пример данных:


,query,answer,answer_document
0,К какому типу относится задача предсказания ку...,Это задача регрессии. Модель предсказывает вещ...,https://education.yandex.ru/handbook/ml/articl...
1,"Дана задача на стилизацию текста. Например, пе...",Это задача генерации новых объектов на основе ...,https://education.yandex.ru/handbook/ml/articl...
2,К какому типу задач относитс задача детектиров...,"В зависимости от того, для чего мы детектируем...",https://education.yandex.ru/handbook/ml/articl...
3,Нужно обучить робокота запрыгивать на стол из ...,"Эту задачу можно решать по-разному. Например, ...",https://education.yandex.ru/handbook/ml/articl...
4,"Поиск наборов товаров, которые посетители супе...",Это задача обучения без учителя.,https://education.yandex.ru/handbook/ml/articl...


In [6]:
predicted_data = pd.read_excel('../data/PredictionsWithMetrics_gigachat_good_2.xlsx')

In [7]:
required_columns = ["query", "answer", "answer_document"]
if not all(col in data.columns for col in required_columns):
    raise ValueError(f"В файле должны быть колонки: {required_columns}")


In [ ]:
embeddings = BGEEmbeddings()

if os.path.exists(index_path):
    vectorstore = FAISS.load_local(
        index_path, embeddings, allow_dangerous_deserialization=True)
else:
    documents = parse_html_with_langchain(data_dir)
    split_docs = RecursiveCharacterTextSplitter(
        chunk_size=1000, chunk_overlap=200).split_documents(documents)
    vectorstore = create_vectorstore(split_docs)
    vectorstore.save_local(index_path)

retriever = vectorstore.as_retriever(
    search_type="similarity", search_kwargs={"k": 5})

# Настройка LLM и цепочки RAG
llm = get_llm()
rag_chain = create_rag_chain(retriever, llm)

# Формирование результатов и расчет метрик
results = []

for _, row in data.iterrows():
    query = row["query"]
    ground_truth = row["answer"]

    # Получение ответа
    response = rag_chain.invoke({"input": query})
    predicted_answer = response["answer"]

    # Получение релевантных документов
    retrieved_docs = retriever.get_relevant_documents(query)
    retrieved_texts = " || ".join([doc.page_content for doc in retrieved_docs])

    # Метрики
    bleu = sentence_bleu([ground_truth.split()], predicted_answer.split())
    scorer = rouge_scorer.RougeScorer(
        ["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    rouge = scorer.score(ground_truth, predicted_answer)
    rouge1 = rouge["rouge1"].fmeasure
    rouge2 = rouge["rouge2"].fmeasure
    rougeL = rouge["rougeL"].fmeasure
    P, R, F1 = score([predicted_answer], [ground_truth],
                     lang="en", verbose=False)
    bertscore = F1.mean().item()

    # Сохранение результатов
    results.append({
        "query": query,
        "ground_truth": ground_truth,
        "predicted_answer": predicted_answer,
        "retrieved_documents": retrieved_texts,
        "bleu": bleu,
        "rouge1": rouge1,
        "rouge2": rouge2,
        "rougeL": rougeL,
        "bertscore": bertscore
    })

results_df = pd.DataFrame(results)
results_df.to_csv("../data/RAG_EvaluationResults.csv", index=False)

In [ ]:
results_df = pd.read_csv('../data/RAG_EvaluationResults.csv',header=1)
results_df

,query,ground_truth,predicted_answer,bleu,rouge1,rouge2,rougeL,bertscore
0,К какому типу относится задача предсказания ку...,Это задача регрессии. Модель предсказывает вещ...,Задача предсказания курса евро к доллару на сл...,0.000000e+00,0.000000,0.000000,0.000000,0.903012
1,"Дана задача на стилизацию текста. Например, пе...",Это задача генерации новых объектов на основе ...,Эта задача относится к **задаче трансляции** (...,7.784451e-232,0.000000,0.000000,0.000000,0.890877
2,К какому типу задач относитс задача детектиров...,"В зависимости от того, для чего мы детектируем...",Это задача **классификации**. Мы пытаемся опре...,4.402213e-232,0.000000,0.000000,0.000000,0.909253
3,Нужно обучить робокота запрыгивать на стол из ...,"Эту задачу можно решать по-разному. Например, ...",Это задача обучения с подкреплением (reinforce...,1.345468e-11,1.000000,1.000000,1.000000,0.867115
4,"Поиск наборов товаров, которые посетители супе...",Это задача обучения без учителя.,Эта задача относится к **коллаборативной фильт...,7.156423e-232,0.000000,0.000000,0.000000,0.879016
5,Что такое машинное обучение?,"Машинное обучение — это наука, изучающая алгор...","Машинное обучение — это наука, изучающая алгор...",5.491005e-01,0.000000,0.000000,0.000000,0.984180
6,Чем занимался Алан Тьюринг в контексте создани...,Когда Алан Тьюринг работал над первыми (компью...,Алан Тьюринг работал над созданием первых комп...,3.790326e-01,0.000000,0.000000,0.000000,0.964707
7,"Какую задачу решали люди, когда пытались расши...",Поиск расшифровки требовал перебора массы вари...,"Люди пытались расшифровать сообщения, закодиро...",1.000369e-231,0.000000,0.000000,0.000000,0.921617
8,Что такое NP-трудные задачи?,"NP-трудные задачи — класс задач, которые нельз...","NP-трудные задачи — это класс задач, решение к...",1.635746e-78,1.000000,0.000000,1.000000,0.910000
9,"Какие задачи сложно запрограммировать, но легк...",Например: перевести текст с одного языка на др...,"Задачи, связанные с **оценкой визуальной инфор...",7.074154e-232,0.000000,0.000000,0.000000,0.913681


In [13]:
print ('Mean BertScore - ',results_df['bertscore'].mean())
print ('Mean RougeL - ',results_df['rougeL'].mean())
print ('Mean BLEU - ',results_df['bleu'].mean())


Mean BertScore -  0.9165961510044033
Mean RougeL -  0.2073715361850955
Mean BLEU -  0.06915476860606926
